Öncelikle içerisinde büyük bir veri ile eğitilmiş bir çok modeli barındıran sentence-transformers 'ı indiriyoruz. Daha çok bilgi için lütfen bknz: https://www.sbert.net/docs/pretrained_models.html#question-answer-retrieval-msmarco

In [ ]:
!pip install -U sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/nq-distilbert-base-v1')
embeddings = model.encode(sentences)
print(embeddings)
print(len(embeddings))

[[-0.00650205  0.0116781  -0.46286413 ... -0.10343838 -0.54856056
  -0.31966949]
 [ 0.00813697  0.07245736  0.16088133 ... -0.01075379 -0.37492546
  -0.82051414]]
2


Cümle-dönüştürücüler olmadan , modeli şu şekilde kullanabilirsiniz: İlk olarak, girişinizi dönüştürücü modelinden geçirirsiniz, ardından bağlamsallaştırılmış kelime gömmelerinin üzerinde doğru havuzlama-işlemini uygulamanız gerekir.

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Doğru ortalama için dikkat maskesini hesaba katın
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #model_output öğesinin ilk öğesi, tüm belirteç katıştırmalarını içerir
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Cümle yerleştirmelerini istediğimiz cümleler
sentences = ['This is an example sentence', 'Each sentence is converted']

# HuggingFace Hub'dan modeli yükleyin
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/nq-distilbert-base-v1')
model = AutoModel.from_pretrained('sentence-transformers/nq-distilbert-base-v1')

# Cümleleri belirt
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Hesaplama belirteci yerleştirmeleri
with torch.no_grad():
    model_output = model(**encoded_input)

# Havuzlama gerçekleştirin. Bu durumda, maksimum havuzlama.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[-0.0065,  0.0117, -0.4629,  ..., -0.1034, -0.5486, -0.3197],
        [ 0.0081,  0.0725,  0.1609,  ..., -0.0108, -0.3749, -0.8205]])


sentence-transformers kullanımı

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

#cümlelerimiz.
sentences = ['Mevcudiyetinin ve istikbalinin yegâne temeli budur.',
    'Bu temel, senin en kıymetli hazinendir.',
    'İstikbalde dahi seni bu hazineden mahrum etmek isteyecek dâhilî ve haricî bedhahların olacaktır.',
    'Bir gün, istiklal ve cumhuriyeti müdafaa mecburiyetine düşersen, vazifeye atılmak için içinde bulunacağın vaziyetin imkân ve şeraitini düşünmeyeceksin.']

#cümleler model.encode() çağrılarak kodlanır.
embeddings = model.encode(sentences)

#cümlelerin embeddings'lerini yazdıralım.
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")
    print(len(embedding))

## Nutuk Örneği

In [7]:
model = SentenceTransformer('nq-distilbert-base-v1')
text = ''' Ey Türk gençliği! Birinci vazifen; Türk istiklalini, Türk cumhuriyetini, ilelebet muhafaza ve müdafaa etmektir.
Mevcudiyetinin ve istikbalinin yegâne temeli budur. Bu temel, senin en kıymetli hazinendir. İstikbalde dahi seni bu hazineden mahrum etmek isteyecek dâhilî ve haricî bedhahların olacaktır. Bir gün, istiklal ve cumhuriyeti müdafaa mecburiyetine düşersen, vazifeye atılmak için içinde bulunacağın vaziyetin imkân ve şeraitini düşünmeyeceksin. Bu imkân ve şerait, çok namüsait bir mahiyette tezahür edebilir. İstiklal ve cumhuriyetine kastedecek düşmanlar, bütün dünyada emsali görülmemiş bir galibiyetin mümessili olabilirler. Cebren ve hile ile aziz vatanın bütün kaleleri zapt edilmiş, bütün tersanelerine girilmiş, bütün orduları dağıtılmış ve memleketin her köşesi bilfiil işgal edilmiş olabilir. Bütün bu şeraitten daha elim ve daha vahim olmak üzere, memleketin dâhilinde iktidara sahip olanlar, gaflet ve dalalet ve hatta hıyanet içinde bulunabilirler. Hatta bu iktidar sahipleri, şahsi menfaatlerini müstevlilerin siyasi emelleriyle tevhit edebilirler. Millet, fakruzaruret içinde harap ve bitap düşmüş olabilir.Ey Türk istikbalinin evladı! İşte, bu ahval ve şerait içinde dahi vazifen, Türk istiklal ve cumhuriyetini kurtarmaktır. Muhtaç olduğun kudret, damarlarındaki asil kanda mevcuttur.
'''

In [8]:
from sentence_transformers import SentenceTransformer,util

In [9]:
def extractSentencesFromText(text):
    sections = text.replace('\n',' ').replace('\r',' ').split('.')
    passages = []
    for section in sections:
        passages.append(['', removeUnicode(section.strip())])
    return passages

def removeUnicode(text):
    string_encode = text.encode("ascii", "ignore")
    return string_encode.decode()

def scorePassages(query):
    query_embedding = model.encode(query)
    scores = util.pytorch_cos_sim(query_embedding, passage_embedding).tolist()[0]
    score_dict = {}
    sentence_counter = 0
    for score in scores:
        score_dict[sentence_counter] = score
        sentence_counter+=1
    return score_dict

In [10]:
passages = extractSentencesFromText(text)
print ('Toplam Cümle Sayısı:', len(passages))
passages[:3]

Toplam Cümle Sayısı: 14


[['',
  'Ey Trk genlii! Birinci vazifen; Trk istiklalini, Trk cumhuriyetini, ilelebet muhafaza ve mdafaa etmektir'],
 ['', 'Mevcudiyetinin ve istikbalinin yegne temeli budur'],
 ['', 'Bu temel, senin en kymetli hazinendir']]

In [11]:
passage_embedding = model.encode(passages)
print ('Cümlelerin 768 lik Embeddingsleri:')
print (passage_embedding[:3])

Cümlelerin 768 lik Embeddingsleri:
[[ 0.50167346 -0.2536147   0.5136146  ... -0.01718979 -0.62043864
   0.09258723]
 [ 1.0285922  -0.22050168 -0.21406339 ...  0.14529671  0.37249345
   0.285854  ]
 [ 0.33517528 -0.07789075  0.04679334 ...  0.27558494 -0.7144708
   0.04169514]]


In [14]:
#query = "muhtaç olduğum kudret nerededir?"
#query = "senin en kıymetli hazinen?"
query = "birinci vazifen?"

score_dict = scorePassages(query)

for w in sorted(score_dict, key=score_dict.get, reverse=True):
    print(score_dict[w], passages[w][1])

0.4577587842941284 Ey Trk genlii! Birinci vazifen; Trk istiklalini, Trk cumhuriyetini, ilelebet muhafaza ve mdafaa etmektir
0.4443485140800476 Millet, fakruzaruret iinde harap ve bitap dm olabilir
0.3843265473842621 Bu temel, senin en kymetli hazinendir
0.3623889088630676 Cebren ve hile ile aziz vatann btn kaleleri zapt edilmi, btn tersanelerine girilmi, btn ordular datlm ve memleketin her kesi bilfiil igal edilmi olabilir
0.35933542251586914 stikbalde dahi seni bu hazineden mahrum etmek isteyecek dhil ve haric bedhahlarn olacaktr
0.35218799114227295 Hatta bu iktidar sahipleri, ahsi menfaatlerini mstevlilerin siyasi emelleriyle tevhit edebilirler
0.3363102376461029 Bir gn, istiklal ve cumhuriyeti mdafaa mecburiyetine dersen, vazifeye atlmak iin iinde bulunacan vaziyetin imkn ve eraitini dnmeyeceksin
0.3285766839981079 Muhta olduun kudret, damarlarndaki asil kanda mevcuttur
0.3279009759426117 Mevcudiyetinin ve istikbalinin yegne temeli budur
0.32097771763801575 Ey Trk istikbalinin evlad